<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Reduce Operators

Sometimes we need to _collapse_ a dimension of our datacube.

Reduce the temporal dimension to a single value, the mean for instance:

In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"
spatial_extent = {"west": 11.259613, "east": 11.406212, "south": 46.461019, "north": 46.522237}
temporal_extent = ["2022-05-10T00:00:00Z","2022-06-30T00:00:00Z"]
bands = ["red","green","blue"]
datacube = local_conn.load_stac(url=url,
                                spatial_extent=spatial_extent,
                                temporal_extent=temporal_extent,
                                bands=bands)

datacube_min_time = datacube.reduce_dimension(dimension="time",reducer="mean")
datacube_min_time

Check what happens to the datacube inspecting the resulting xArray object:

In [2]:
datacube_min_time.execute()

<xarray.DataArray 'stackstac-909439cd47570b0d75bd2f7623072b18' (band: 3,
                                                                y: 713, x: 1145)>
dask.array<mean_agg-aggregate, shape=(3, 713, 1145), dtype=float64, chunksize=(1, 606, 860), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * band                                     (band) <U5 'red' 'green' 'blue'
  * x                                        (x) float64 6.733e+05 ... 6.848e+05
  * y                                        (y) float64 5.155e+06 ... 5.148e+06
    instruments                              <U3 'msi'
    mgrs:grid_square                         <U2 'PS'
    processing:software                      object {'sentinel2-to-stac': '0....
    ...                                       ...
    proj:shape                               object {10980}
    proj:transform                           object {0, 600000, 10, 5200020, ...
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int32 32632
Attributes:
    spec:                           RasterSpec(epsg=32632, bounds=(600000.0, ...
    crs:                            epsg:32632
    transform:                      | 10.00, 0.00, 600000.00|\n| 0.00,-10.00,...
    resolution:                     10.0
    reduced_dimensions_min_values:  {'time': numpy.datetime64('2022-05-11T10:...

It is possible to reduce in the same way all the available dimensions of the datacube.

Additionally, it is possible to reduce both spatial dimensions of the datacube at the same time.
To do this, we need the `reduce_spatial` process.
We show an example where it is used to compute the standard deviation `sd`.

In [3]:
datacube_spatial_sd = datacube.reduce_spatial(reducer="sd")
datacube_spatial_sd

Verify that the spatial dimensions were collapsed:

In [4]:
datacube_spatial_sd.execute()

<xarray.DataArray 'stackstac-a73678c210b830604cc58af19ec97d68' (time: 21,
                                                                band: 3)>
dask.array<_sqrt, shape=(21, 3), dtype=float64, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 2022-05-11...
    id                                       (time) <U24 'S2A_32TPS_20220511_...
  * band                                     (band) <U5 'red' 'green' 'blue'
    instruments                              <U3 'msi'
    mgrs:grid_square                         <U2 'PS'
    s2:mgrs_tile                             (time) object '32TPS' ... '32TPS'
    ...                                       ...
    proj:shape                               object {10980}
    proj:transform                           object {0, 600000, 10, 5200020, ...
    common_name                              (band) <U5 'red' 'green' 'blue'
    center_wavelength                        (band) float64 0.665 0.56 0.49
    full_width_half_max                      (band) float64 0.038 0.045 0.098
    epsg                                     int32 32632
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 5090220.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5200020.00|\n| 0.0...
    resolution:  10.0